In [ ]:
# Step 1. Data Prep

In [1]:
import os
import re
import pandas as pd
from cyvcf2 import VCF
from tqdm import tqdm

# Base directory for ClinVar data
BASE_DIR = "./clinvar_data"
os.makedirs(BASE_DIR, exist_ok=True)

# Filepaths
CLINVAR_VCF_PATH = f"{BASE_DIR}/clinvar_GRCh38_latest.vcf.gz"
CLINVAR_TBI_PATH = f"{CLINVAR_VCF_PATH}.tbi"

os.environ["CLINVAR_DIR"] = BASE_DIR
os.environ["CLINVAR_VCF"] = CLINVAR_VCF_PATH

print("ClinVar directory:", BASE_DIR)
print("ClinVar VCF path:", CLINVAR_VCF_PATH)

ClinVar directory: ./clinvar_data
ClinVar VCF path: ./clinvar_data/clinvar_GRCh38_latest.vcf.gz


In [2]:
# Download ClinVar VCF (GRCh38) with curl
!curl -L \
  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz \
  -o $CLINVAR_VCF_PATH

# Download index (.tbi)
!curl -L \
  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz.tbi \
  -o $CLINVAR_TBI_PATH

print("Downloaded ClinVar VCF and index using curl.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  172M  100  172M    0     0  99.0M      0  0:00:01  0:00:01 --:--:-- 99.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  590k  100  590k    0     0  7386k      0 --:--:-- --:--:-- --:--:-- 7386k
Downloaded ClinVar VCF and index using curl.


In [20]:
def parse_clinvar_strict(vcf_path: str, sample_uncertain_k: int = 100_000):
    """
    STRICT ClinVar parser:
      • SNVs only
      • Accept *only* pure Pathogenic or pure Benign
      • Exclude: Likely Pathogenic/Benign, Uncertain, Conflicting, mixed
    """
    vcf = VCF(vcf_path)
    benign, pathogenic, uncertain = [], [], []

    def normalize_sig(val: str) -> list[str]:
        toks = re.split(r"[\|;,/]+", val)
        out = []
        for t in toks:
            t = t.strip().lower().replace(" ", "_").replace("-", "_")
            if t:
                out.append(t)
        return out

    for var in tqdm(vcf, desc="Parsing ClinVar VCF (strict)"):
        # SNVs only
        if len(var.REF) != 1 or var.ALT is None or len(var.ALT) != 1 or any(len(a) != 1 for a in var.ALT):
            continue

        clnsig = var.INFO.get("CLNSIG")
        if clnsig is None:
            continue

        toks = set(normalize_sig(str(clnsig)))

        # Clinical significance flags
        has_path = "pathogenic" in toks
        has_ben  = "benign" in toks
        has_lpath = "likely_pathogenic" in toks
        has_lben  = "likely_benign" in toks
        has_unc   = "uncertain_significance" in toks
        has_conf  = "conflicting_interpretations_of_pathogenicity" in toks

        # STRICT filtering: drop uncertain, conflicting, likely*
        if has_conf or has_unc or has_lpath or has_lben:
            continue

        # Drop mixed pathogenic+benign
        if has_path and has_ben:
            continue

        # Accept pure classes only
        if has_path and not has_ben:
            pathogenic.append(var)
            continue
        if has_ben and not has_path:
            benign.append(var)
            continue

        # All other mixed/unhandled cases are dropped

    print(f"STRICT collection complete → benign={len(benign)}, pathogenic={len(pathogenic)}")
    return benign, pathogenic, uncertain

In [21]:
# Run parser and convert results into a DataFrame
benign_vars, pathogenic_vars, _ = parse_clinvar_strict(CLINVAR_VCF_PATH)

def to_dataframe(vars_list, label):
    rows = []
    for v in vars_list:
        rows.append({
            "CHROM": v.CHROM,
            "POS": v.POS,
            "REF": v.REF,
            "ALT": v.ALT[0],
            "CLNSIG": label
        })
    return pd.DataFrame(rows)

df_benign = to_dataframe(benign_vars, "Benign")
df_pathogenic = to_dataframe(pathogenic_vars, "Pathogenic")

df = pd.concat([df_benign, df_pathogenic], ignore_index=True)

print("Final strict dataset size:", len(df))
df.head()

Parsing ClinVar VCF (strict): 4125815it [00:50, 80944.87it/s]


STRICT collection complete → benign=180304, pathogenic=82205
Final strict dataset size: 262509


,CHROM,POS,REF,ALT,CLNSIG
0,1,930165,G,A,Benign
1,1,930204,G,A,Benign
2,1,930285,G,A,Benign
3,1,930314,C,T,Benign
4,1,930325,C,T,Benign


In [22]:
# Save dataset 
output_csv = f"{BASE_DIR}/clinvar_snv_pathogenic_benign_STRICT.csv"
df.to_csv(output_csv, index=False)

print("Saved strict dataset to:", output_csv)

Saved strict dataset to: ./clinvar_data/clinvar_snv_pathogenic_benign_STRICT.csv


In [3]:
# Step 2. Feature extraction

In [6]:
# Annotation-based Features

# Feature 1: CADD
# Could use this as a baseline only. Evaluate this versus adding additional annotation/sequence based features to a model

import os
import pandas as pd
from tqdm import tqdm
import pysam

CADD_DIR = "./cadd_data"
os.makedirs(CADD_DIR, exist_ok=True)

CADD_TSV = f"{CADD_DIR}/gnomad_snv_cadd.tsv.gz"
CADD_TBI = f"{CADD_TSV}.tbi"

print("CADD will be stored in:", CADD_DIR)



CADD will be stored in: ./cadd_data


In [ ]:
# Let's skip the whole genome (300GB uncompressed) for now
#import os

#CADD_DIR = "./cadd_data"
#os.makedirs(CADD_DIR, exist_ok=True)

#CADD_TSV = f"{CADD_DIR}/CADD_GRCh38_v1.6.tsv.gz"
#CADD_TBI = f"{CADD_TSV}.tbi"

# Download CADD SNV scores for GRCh38
#!curl -L \
#  https://krishna.gs.washington.edu/download/CADD/v1.6/GRCh38/whole_genome_SNVs.tsv.gz \
#  -o $CADD_TSV

#!curl -L \
#  https://krishna.gs.washington.edu/download/CADD/v1.6/GRCh38/whole_genome_SNVs.tsv.gz.tbi \
#  -o $CADD_TBI

#print("Downloaded CADD v1.6 GRCh38 SNV scores.")

In [5]:
# gnomAD-based CADD SNV file (≈6GB)
# May miss some rare variants. Could use this versus whole genome as a comparison for final project
!curl -L \
  https://krishna.gs.washington.edu/download/CADD/v1.6/GRCh38/gnomad.genomes.r3.0.snv.tsv.gz \
  -o $CADD_TSV

!curl -L \
  https://krishna.gs.washington.edu/download/CADD/v1.6/GRCh38/gnomad.genomes.r3.0.snv.tsv.gz.tbi \
  -o $CADD_TBI

print("Downloaded gnomAD-based CADD SNV file (v1.6, GRCh38).")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6058M  100 6058M    0     0  38.0M      0  0:02:39  0:02:39 --:--:-- 41.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2503k  100 2503k    0     0  3419k      0 --:--:-- --:--:-- --:--:-- 3414k
Downloaded gnomAD-based CADD SNV file (v1.6, GRCh38).


In [8]:
# Load ClinVar
df = pd.read_csv("./clinvar_data/clinvar_snv_pathogenic_benign_STRICT.csv")
print("Loaded ClinVar SNV dataset:", df.shape)

Loaded ClinVar SNV dataset: (262509, 5)


/tmp/ipykernel_3995140/198660698.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./clinvar_data/clinvar_snv_pathogenic_benign_STRICT.csv")


In [10]:
# CADD Lookup Function

# CADD TSV format:
# Chrom  Pos  Ref  Alt  RawScore  PHRED

# Use tabix to fetch rows efficiently
cadd_tabix = pysam.TabixFile(CADD_TSV)

def get_cadd_score(chrom, pos, ref, alt, tabix_obj):
    """
    Fetch CADD raw + PHRED scores for a single SNV.
    Works with gnomAD CADD SNV TSV files.
    """
    try:
        # CADD uses 1-based chromosome indexing
        region = tabix_obj.fetch(chrom, pos-1, pos)
    except ValueError:
        return None, None

    for row in region:
        fields = row.split("\t")
        c_ref = fields[2]
        c_alt = fields[3]
        if c_ref == ref and c_alt == alt:
            raw = float(fields[4])
            phred = float(fields[5])
            return raw, phred

    return None, None

In [ ]:
ref_genome

In [ ]:
# ref_genome.keys()

In [ ]:
# vcf.seqnames[:10]

In [ ]:
# Map RefSeq NC_* accessions to Ensembl-like chrom labels when possible
NC_TO_ENSEMBL_SPECIAL = {
    23: "X",
    24: "Y",
    12920: "MT",  # NC_012920.1 → mitochondrion
}


In [ ]:
def contig_candidates(chrom: str) -> List[str]:
    """Return plausible contig name candidates across naming conventions.
    - Accepts inputs like 'chr1', '1', 'NC_000001.11', 'chrX', 'X', 'MT', 'chrM'
    - Returns unique candidates in priority order.
    """
    cands = []
    # As-is
    cands.append(chrom)

    # Strip/add 'chr'
    if chrom.startswith("chr"):
        cands.append(chrom[3:])
    else:
        cands.append("chr" + chrom)

    # Map NC_0000XX.yy → 1..22/X/Y/MT
    m = re.match(r"NC_(\d{6})\.(\d+)", chrom)
    if m:
        num = int(m.group(1))
        if 1 <= num <= 22:
            cands += [str(num), f"chr{num}"]
        elif num in NC_TO_ENSEMBL_SPECIAL:
            val = NC_TO_ENSEMBL_SPECIAL[num]
            cands += [val, f"chr{val}"]

    # Support chrM/chrMT → MT
    if chrom in ("chrM", "chrMT"):
        cands += ["MT"]

    # Deduplicate preserving order
    out = []
    for x in cands:
        if x not in out:
            out.append(x)
    return out

In [12]:
# Save annotated dataset 
output = "./clinvar_data/clinvar_STRICT_with_CADD.csv"
df.to_csv(output, index=False)
print("Saved:", output)

Saved: ./clinvar_data/clinvar_STRICT_with_CADD.csv


In [24]:
# Optional
# Before annotation, let's look at datasets.  
import pandas as pd
import pysam

# ---- Load ClinVar ----
df = pd.read_csv("./clinvar_data/clinvar_snv_pathogenic_benign_STRICT.csv")

print("=== First 5 ClinVar rows ===")
print(df[["CHROM", "POS", "REF", "ALT", "CLNSIG"]].head())

# ---------------------------
# Try to find matching CADD rows 
# for the first 5 ClinVar variants
# ---------------------------
print("\n=== Matching CADD rows for first 5 ClinVar variants ===")

def lookup_cadd(chrom, pos, ref, alt):
    try:
        for r in cadd_tabix.fetch(str(chrom), pos-1, pos):
            fields = r.split("\t")
            c_ref, c_alt = fields[2], fields[3]
            if c_ref == ref and c_alt == alt:
                return r
        return None
    except Exception as e:
        return None

for idx in range(5):
    row = df.iloc[idx]
    chrom, pos, ref, alt = row["CHROM"], int(row["POS"]), row["REF"], row["ALT"]

    print(f"\nClinVar #{idx}: CHROM={chrom} POS={pos} REF={ref} ALT={alt}")
    cadd_row = lookup_cadd(chrom, pos, ref, alt)

    if cadd_row:
        print("Matched CADD row:", cadd_row)
    else:
        print("No matching CADD row found.")

# ---- Load CADD Tabix ----
cadd_tabix = pysam.TabixFile("./cadd_data/gnomad_snv_cadd.tsv.gz")

print("\n=== First 5 CADD rows ===")
import gzip

with gzip.open("./cadd_data/gnomad_snv_cadd.tsv.gz", "rt") as f:
    for _ in range(5):
        print(f.readline().strip())


=== First 5 ClinVar rows ===
  CHROM     POS REF ALT  CLNSIG
0     1  930165   G   A  Benign
1     1  930204   G   A  Benign
2     1  930285   G   A  Benign
3     1  930314   C   T  Benign
4     1  930325   C   T  Benign

=== Matching CADD rows for first 5 ClinVar variants ===

ClinVar #0: CHROM=1 POS=930165 REF=G ALT=A
Matched CADD row: 1	930165	G	A	4.225333	28.9

ClinVar #1: CHROM=1 POS=930204 REF=G ALT=A
Matched CADD row: 1	930204	G	A	2.879554	23.2

ClinVar #2: CHROM=1 POS=930285 REF=G ALT=A
Matched CADD row: 1	930285	G	A	0.091394	2.015

ClinVar #3: CHROM=1 POS=930314 REF=C ALT=T
Matched CADD row: 1	930314	C	T	2.481464	22.4

ClinVar #4: CHROM=1 POS=930325 REF=C ALT=T
Matched CADD row: 1	930325	C	T	1.053778	12.28

=== First 5 CADD rows ===
## CADD GRCh38-v1.6 (c) University of Washington, Hudson-Alpha Institute for Biotechnology and Berlin Institute of Health 2013-2020. All rights reserved.
#Chrom	Pos	Ref	Alt	RawScore	PHRED
1	10031	T	C	0.756535	8.973
1	10037	T	C	0.748894	8.902
1	1004

/tmp/ipykernel_3995140/2302147126.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./clinvar_data/clinvar_snv_pathogenic_benign_STRICT.csv")


In [11]:
# Annotate ClinVar dataset with CADD

cadd_raw = []
cadd_phred = []

for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Annotating CADD"):
    raw, phred = get_cadd_score(
        str(row["CHROM"]),
        int(row["POS"]),
        row["REF"],
        row["ALT"],
        cadd_tabix
    )
    cadd_raw.append(raw)
    cadd_phred.append(phred)

df["CADD_raw"] = cadd_raw
df["CADD_phred"] = cadd_phred

Annotating CADD: 100%|██████████| 262509/262509 [05:06<00:00, 856.43it/s] 


In [13]:
# Step 3: Train model 

# Load CADD-annotated dataset
import pandas as pd

df = pd.read_csv("./clinvar_data/clinvar_STRICT_with_CADD.csv")
print(df.shape)
df.head()

# Clean dataset (drop missing CADD scores)
df_clean = df.dropna(subset=["CADD_raw", "CADD_phred"]).copy()
print("After dropping missing values:", df_clean.shape)

# Encode labels 
#  Pathogenic is 1
#  Benign is 0 

df_clean["label"] = (df_clean["CLNSIG"] == "Pathogenic").astype(int)

# Choose Features. Let's start with CADD_phred only
# "How good is CADD PHRED at distinguishing Pathogenic vs Benign?"
X = df_clean[["CADD_phred"]].values
y = df_clean["label"].values

(262509, 7)
After dropping missing values: (181036, 7)


/tmp/ipykernel_3995140/744477208.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./clinvar_data/clinvar_STRICT_with_CADD.csv")


In [14]:
# Logistic Regression Baseline 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# Metrics
auc = roc_auc_score(y_test, y_proba)
print("Logistic Regression AUROC:", auc)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Logistic Regression AUROC: 0.983308543201827

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     34171
           1       0.90      0.73      0.81      2037

    accuracy                           0.98     36208
   macro avg       0.94      0.86      0.90     36208
weighted avg       0.98      0.98      0.98     36208



In [16]:
# XGBoost Baseline 
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

model.fit(X_train, y_train)

y_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_proba > 0.5).astype(int)

auc = roc_auc_score(y_test, y_proba)
print("XGBoost AUROC:", auc)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

XGBoost AUROC: 0.9824731809221883

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     34171
           1       0.89      0.74      0.81      2037

    accuracy                           0.98     36208
   macro avg       0.94      0.87      0.90     36208
weighted avg       0.98      0.98      0.98     36208

